# Chatbot

In [2]:
!pip install python-utils

  Obtaining dependency information for python-utils from https://files.pythonhosted.org/packages/f0/7b/e83e7b184e53530abe064b237a3731c738d3cb59f4201f3ce1a4ec0efe6f/python_utils-3.8.1-py2.py3-none-any.whl.metadata
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import os
import openai
import sys
import time
import pandas as pd
import numpy as np
import psycopg2
sys.path.append('../..')
import python_utils
# from dotenv import loader_dotenv, find_dotenv

## Helper Functions

In [11]:
# This function receives an array of 'messages' and returns an output based on this array.
# INPUTS:
#      * messages: an array of messages between user and gpt model.
#      * TODO: complete
def get_completion_from_messages(messages, model="gpt-3.5-turbo-16k", temperature=0, max_tokens=500):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

# TODO: Complete this funciton
def get_prompt_category(user_prompt):
    return None


# TODO: Complete this funciton
def get_content_from_database(user_prompt):
    return None

## Chatbot without Ambiguous Query Processing

In [12]:
panels = []
delimiter = "~~~~"
start_convo_flag = True

context = [{'role': 'system', 'content':f"""

Just be a normal chatbot.

"""}] # accumulate messages


context.append({'role':'user', 'content':f"{delimiter}Hi!{delimiter}"})
init_response = get_completion_from_messages(context)

print("-- Chatbot: ", init_response, "\n")
while start_convo_flag:
    print("-- User: ")
    user_input = input()
    print()
    if user_input == "stop":
        break
        
    # Check if conversation is a continuation of a particular topic
    # conversation_history = [entry for entry in context if entry['role'] != 'system']
    # continuation_flag = int(is_continuation(conversation_history, user_input)
    
    # Match question with NQ Dataset
    db_content = get_content_from_database(user_input)
    
    # Add System instructions
    context.append({'role':'system', 'content': f"""Use the provided content to answer the question.\
                    Do not make assumptions or use information from your pre-trained data. Use this information: {db_content}"""})
    
    # Input User's question
    context.append({'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"})
    response = get_completion_from_messages(context)
    print("-- Chatbot: ", response, "\n")
    context.append({'role': 'assistant', 'content': f"{response}"})
    



OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## Chatbot With Ambiguous Query Processing